In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
zip_folder = '/content/drive/My Drive/Deep Learning Machine Learning/data.zip'
extract_folder = '/content/drive/My Drive/Deep Learning Machine Learning/data/'

In [ ]:
import cv2 as cv
import os
from PIL import Image

from torch import nn

from torch.utils.data import DataLoader, Dataset
from torchvision import transforms

import matplotlib.pyplot as plt
import numpy as np
import torch

from torch.utils.tensorboard import SummaryWriter

import torch.optim as optim

In [ ]:
import os
import zipfile
import pandas as pd

def extract_zip(zip_path, extract_path):
      if not os.path.exists(extract_path):
          os.makedirs(extract_path)
      with zipfile.ZipFile(zip_path, 'r') as zip_ref:
          zip_ref.extractall(extract_path)

In [ ]:
data_path = os.path.join(extract_folder + "data")
data_dir = os.listdir(data_path)
print(data_dir)

items = os.listdir(os.path.join((extract_folder + "data"), "train"))
label_dict = {category: idx for idx, category in enumerate(items)}
print(label_dict)

['test', 'train', 'valid']
{'accessories': 0, 'jackets': 1, 'jeans': 2, 'knitwear': 3, 'shirts': 4, 'shoes': 5, 'shorts': 6, 'tees': 7}


In [ ]:
class Data_class(Dataset):
    def __init__(self, label_map, set_path, labels=None, transform=None, test=False):

      self.label_map = label_map
      self.labels = labels
      self.set_path = set_path

      self.transform = transform

      self.test = test
      self.data = self.data_loader()

    def data_loader(self):
      images = []

      if self.test:
        img_files = os.listdir(self.set_path)
        for img_file in img_files:
          print(f"loading image {img_file}...")
          img_path = os.path.join(self.set_path, img_file)
          try:
            img = Image.open(img_path).convert('RGB')
            images.append(img)
          except Exception as e:
            print(f"Error loading image {img_path}: {e}")
        return images

      else:
        for item in self.labels:
          category_path = os.path.join(self.set_path, item)

          label = self.label_map[item]
          print(f'Loading {item}...')

          for img_file in os.listdir(category_path):
            img_path = os.path.join(category_path, img_file)

            try:
              img = Image.open(img_path).convert('RGB')

              if self.test == False:
                images.append((img, label))
              else:
                images.append(img)

            except Exception as e:
              print(f"Error loading image {img_path}: {e}")
        return images

    def __len__(self):
      return len(self.data)

    def __getitem__(self, idx):
      if(self.test):
        image = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image

      else:
        image, label = self.data[idx]
        if self.transform:
            image = self.transform(image)
        return image, label

In [ ]:
class Trainer:
  def __init__(self, model, train_loader, val_loader, criterion, optimizer, num_epochs=20):

    #training device variables group
    if torch.cuda.is_available():
      self.device = torch.device('cuda')
    else:
      self.device = torch.device('cpu')
    self.model = model.to(self.device)

    #data loader variables group
    self.train_loader = train_loader
    self.val_loader = val_loader

    #training variables group
    self.criterion = criterion
    self.optimizer = optimizer
    self.num_epochs = num_epochs

    #variabels for tracking and plotting
    self.train_losses = []
    self.val_losses = []
    self.train_accuracies = []
    self.val_accuracies = []

    # TensorBoard writer
    self.writer = SummaryWriter()

  def fit(self):
    for epoch in range(self.num_epochs):

      train_loss, train_accuracy = self.train()
      val_loss, val_accuracy = self.evaluate()

      # Store loss and accuracy values
      self.train_losses.append(train_loss)
      self.train_accuracies.append(train_accuracy)
      self.val_losses.append(val_loss)
      self.val_accuracies.append(val_accuracy)

      # Log metrics to TensorBoard
      self.writer.add_scalar('Loss/Train', train_loss, epoch)
      self.writer.add_scalar('Loss/Validation', val_loss, epoch)
      self.writer.add_scalar('Accuracy/Train', train_accuracy, epoch)
      self.writer.add_scalar('Accuracy/Validation', val_accuracy, epoch)
      print(f'Epoch [{epoch+1}/{self.num_epochs}], '
            f' Train Loss: {train_loss:.4f}, Train Accuracy: {train_accuracy:.2f}%,'
            f' Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy:.2f}%')

      self.writer.close() # Close the writer after training is complete

  def train(self):
    run_loss = 0.0
    correct_pred = 0
    total_samples = 0

    self.model.train()  # Set model to training mode

    for feature, labels in self.train_loader:
      feature, labels = feature.to(self.device), labels.to(self.device)

      self.optimizer.zero_grad()
      outputs = self.model(feature)
      loss = self.criterion(outputs, labels)#next 3 lines are backpropagration step
      loss.backward()
      self.optimizer.step()

      # Track loss and accuracy
      run_loss += loss.item()
      _, predicted = torch.max(outputs, 1)
      correct_pred += (predicted == labels).sum().item()
      total_samples += labels.size(0)

    # Calculate training loss and accuracy
    train_loss = run_loss / len(self.train_loader)
    train_accuracy = correct_pred / total_samples * 100

    return train_loss, train_accuracy

  @torch.no_grad()
  def evaluate(self):
    val_loss = 0.0
    correct_pred = 0
    total_samples = 0

    self.model.eval()  # Set model to evaluation mode
    for images, labels in self.val_loader:
      images, labels = images.to(self.device), labels.to(self.device)

      # Forward pass
      outputs = self.model(images)
      loss = self.criterion(outputs, labels)
      val_loss += loss.item()

      # Track accuracy
      _, predicted = torch.max(outputs, 1)
      correct_pred += (predicted == labels).sum().item()
      total_samples += labels.size(0)

    # Calculate validation loss and accuracy
    val_loss = val_loss / len(self.val_loader)
    val_accuracy = correct_pred / total_samples * 100

    return val_loss, val_accuracy

In [ ]:
import numpy as np


train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(brightness=0.2, contrast=0.2, saturation=0.2, hue=0.2),
    transforms.RandomRotation(20),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

testVal_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

val_set = os.path.join(data_path, "valid")
train_set = os.path.join(data_path, "train")
test_set = os.path.join(data_path, "test/unknown")

print("Loading test set:")
test_dataset = Data_class(label_dict, test_set, transform=testVal_transform, test=True)

print("\nLoading validaiton set: ")
val_dataset = Data_class(label_dict, val_set, items, transform=testVal_transform)

print("\nLoading train set: ")
train_dataset = Data_class(label_dict, train_set, items, transform=train_transform)

print("\n\nFinish uploading")

Loading test set:
loading image test1.jpg...
loading image test2.jpg...
loading image test3.jpg...
loading image test4.jpg...
loading image test5.jpg...
loading image test6.jpg...
loading image test7.jpg...
loading image test8.jpg...

Loading validaiton set: 
Loading accessories...
Loading jackets...
Loading jeans...
Loading knitwear...
Loading shirts...


In [ ]:
batch_size = 32
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print("DataLoader objects created successfully.")

In [ ]:
class MLP(nn.Module):

  def __init__(self, inputSize, outputSize, dropout=0.0):
    super().__init__()

    self.net = nn.Sequential(
        nn.Flatten(),
        nn.Linear(inputSize, 258),
        nn.ReLU(inplace=True),
        nn.Linear(258, 128),
        nn.ReLU(inplace=True),
        nn.Linear(128, 64),
        nn.ReLU(inplace=True),
        nn.Linear(64, outputSize),
        nn.ReLU(inplace=True)
    )

    if torch.cuda.is_available():
        self.cuda()

  def forward(self, X):
    return self.net(X)


In [ ]:
input_dim = 224 * 224 * 3
output_dim = 8

mlp_model = MLP(inputSize=input_dim, outputSize=output_dim, dropout=0.02)

if torch.cuda.is_available():
    mlp_model.cuda()

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(mlp_model.parameters(), lr=1e-4)


mlp_trainer = Trainer(mlp_model, train_dataloader, val_dataloader, criterion, optimizer, num_epochs=15)
mlp_trainer.fit()

In [ ]:
impt matplotlib.pyplot as plt
from matplotlib.ticker import MaxNLocator

def plot_afterTraining(trainer_name, model_name):
  epochs = range(1, trainer_name.num_epochs + 1)
  plt.figure(figsize=(12, 5))

  plt.suptitle(model_name, fontsize=16)

  loss_plot(epochs, trainer_name)
  accuracy_plot(epochs, trainer_name)

  plt.tight_layout()

  #full_save = os.path.join(save_path, model_name + '-curves')
  #plt.savefig(full_save)

  plt.show()
  plt.close()

def loss_plot(epochs, trainer_name):
  plt.subplot(1, 2, 1)
  plt.plot(epochs, trainer_name.train_losses, 'b', label='Training loss')
  plt.plot(epochs, trainer_name.val_losses, 'r', label='Validation loss')
  plt.title('Training and Validation Loss')
  plt.xlabel('Epochs')
  plt.ylabel('Loss')
  plt.legend()

  ax = plt.gca() #get current axis
  ax.xaxis.set_major_locator(MaxNLocator(integer=True)) #only int
  ax.yaxis.set_major_locator(MaxNLocator(nbins=8))


def accuracy_plot(epochs, trainer_name):
  plt.subplot(1, 2, 2)
  plt.plot(epochs, trainer_name.train_accuracies, 'b', label='Training accuracy')
  plt.plot(epochs, trainer_name.val_accuracies, 'r', label='Validation accuracy')
  plt.title('Training and Validation Accuracy')
  plt.xlabel('Epochs')
  plt.ylabel('Accuracy (%)')
  plt.legend()

  ax = plt.gca()
  ax.xaxis.set_major_locator(MaxNLocator(integer=True))
  ax.yaxis.set_major_locator(MaxNLocator(nbins=8))

In [ ]:
plot_afterTraining(mlp_trainer, "mlp_trainer")